In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [2]:
data_1 = pd.read_excel('整理评分.xlsx',sheet_name='红葡萄酒')
data_2 = pd.read_excel('整理评分.xlsx',sheet_name='白葡萄酒')
data_1.head()

,酒样品,第一组评分,第二组评分
0,1,62.7,68.1
1,2,80.3,74.0
2,3,80.4,74.6
3,4,68.6,71.2
4,5,73.3,72.1


## Wilcoxon符号秩和检验

In [3]:
statistic, pvalue = scipy.stats.wilcoxon(data_1['第一组评分'],
                                         data_1['第二组评分'],
                                         zero_method='wilcox',
                                         alternative='two-sided')
print("Wilcoxon Test Statstic:{}   Pvalue:{}".format(statistic,pvalue))

statistic, pvalue = scipy.stats.wilcoxon(data_2['第一组评分'],
                                         data_2['第二组评分'],
                                         zero_method='wilcox',
                                         alternative='two-sided')
print("Wilcoxon Test Statstic:{}   Pvalue:{}".format(statistic,pvalue))

AttributeError: module 'scipy' has no attribute 'stats'

In [ ]:
data_3 = pd.read_excel('整理评分.xlsx',sheet_name='第一组红葡萄酒的评分',header=None).iloc[32:,1:].astype(np.int32)
data_4 = pd.read_excel('整理评分.xlsx',sheet_name='第二组红葡萄酒的评分',header=None).iloc[32:,1:].astype(np.int32)
data_5 = pd.read_excel('整理评分.xlsx',sheet_name='第一组白葡萄酒的评分',header=None).iloc[33:,1:].astype(np.int32)
data_6 = pd.read_excel('整理评分.xlsx',sheet_name='第二组白葡萄酒的评分',header=None).iloc[33:,1:].astype(np.int32)

def func_dis(data):
    data_all = data.iloc[:,:-1].values
    data_mean = data.iloc[:,-1].values.reshape(-1,1)
    temp = (data_all-data_mean)**2
    temp = temp.sum(axis=1)
    temp = np.sqrt(temp).sum()
    return np.round(temp,2)

print('第一组红葡萄酒的总欧氏距离：',func_dis(data_3))
print('第二组红葡萄酒的总欧氏距离：',func_dis(data_4))
print('第一组白葡萄酒的总欧氏距离：',func_dis(data_5))
print('第二组白葡萄酒的总欧氏距离：',func_dis(data_6))

第一组红葡萄酒的总欧氏距离： 504.16
第二组红葡萄酒的总欧氏距离： 513.97
第一组白葡萄酒的总欧氏距离： 666.73
第二组白葡萄酒的总欧氏距离： 937.27


# 问题2
## 相关性分析

In [158]:
quality_red = pd.read_excel('整理评分.xlsx',sheet_name='红葡萄酒')
quality_white = pd.read_excel('整理评分.xlsx',sheet_name='白葡萄酒')
mean_quality_red = quality_red.iloc[:,1:].mean(axis=1)
mean_quality_white = quality_white.iloc[:,1:].mean(axis=1)
index_1 = pd.read_excel('整理指标.xlsx',sheet_name='红葡萄').iloc[:,1:].drop(index=0).reset_index(drop=True)
index_2 = pd.read_excel('整理指标.xlsx',sheet_name='白葡萄').iloc[:,1:].drop(index=0).reset_index(drop=True)
index_1['Sample'] = mean_quality_red
index_2['Sample'] = mean_quality_white
index_1 = index_1.astype(float)
index_2 = index_2.astype(float)

In [137]:
red_sample_corr = index_1.corr(method='pearson')['Sample'].drop(index=['Sample'])
white_sample_corr = index_2.corr(method='pearson')['Sample'].drop(index=['Sample'])

In [140]:
# 红葡萄酒排名前5个指标
red_sample_corr.sort_values(ascending=False)[:5]

葡萄总黄酮         0.683370
DPPH自由基       0.632321
总酚            0.570570
白藜芦醇(mg/L)    0.552959
酒总黄酮          0.530233
Name: Sample, dtype: float64

In [141]:
# 白葡萄酒排名前5个指标
white_sample_corr.sort_values(ascending=False)[:5]

b*（+黄;-蓝）    0.523874
总糖           0.431069
可溶性固形物       0.387984
干物质含量        0.362903
酒石酸          0.361998
Name: Sample, dtype: float64

## Topsis法

In [232]:
def entropy_weight(features):
    features = np.array(features)
    proportion = features / features.sum(axis=0)  # normalized
    entropy = np.nansum(-proportion * np.log(proportion) / np.log(len(features)),
                        axis=0)  # calculate entropy
    weight = (1 - entropy) / (1 - entropy).sum()

    return weight  # calculation weight coefficient


def topsis(data, weight=None):
    # normalized
    data = data / np.sqrt((data ** 2).sum())
    # best and worst solution
    Z = pd.DataFrame([data.min(), data.max()], index=['负理想解', '正理想解'])

    weight = entropy_weight(data) if weight is None else np.array(weight)  # distance
    Result = data.copy()
    Result['正理想解'] = np.sqrt(((data - Z.loc['正理想解']) ** 2 * weight).sum(axis=1))
    Result['负理想解'] = np.sqrt(((data - Z.loc['负理想解']) ** 2 * weight).sum(axis=1))

    # composite score index
    Result['综合得分指数'] = Result['负理想解'] / (Result['负理想解'] + Result['正理想解'])
    Result['排序'] = Result.rank(ascending=False)['综合得分指数']

    return Result, Z, weight

In [242]:
temp_1 = red_sample_corr.sort_values(ascending=False)[:5]
index_temp_1 = temp_1.index.values
topsis_data_1 = index_1[index_temp_1]

temp_2 = white_sample_corr.sort_values(ascending=False)[:5]
index_temp_2 = temp_2.index.values
topsis_data_2 = index_2[index_temp_2]

Result_1, Z_1, weight_result_1 = topsis(topsis_data_1, weight=None)
Result_2, Z_2, weight_result_2 = topsis(topsis_data_2, weight=None)

In [243]:
weight_result_1 # 红葡萄酒的权重

array([0.2035933 , 0.0696713 , 0.12956911, 0.38076335, 0.21640294])

In [244]:
Result_1 # 红葡萄酒的排名

,葡萄总黄酮,DPPH自由基,总酚,白藜芦醇(mg/L),酒总黄酮,正理想解,负理想解,综合得分指数,排序
0,0.191815,0.230029,0.282437,0.101802,0.270467,0.310980,0.152491,0.329020,9.0
1,0.279353,0.248339,0.321574,0.152332,0.448540,0.253663,0.244426,0.490728,4.0
2,0.218419,0.218733,0.259468,0.219019,0.248468,0.251390,0.185886,0.425101,6.0
3,0.090685,0.142015,0.128011,0.122492,0.145227,0.354821,0.081888,0.187512,17.0
4,0.207906,0.211833,0.210804,0.208637,0.122881,0.287900,0.152977,0.346984,8.0
5,0.138374,0.147077,0.127688,0.185012,0.149904,0.316143,0.121272,0.277246,11.0
6,0.070177,0.093887,0.110254,0.076009,0.093260,0.393287,0.042957,0.098470,25.0
7,0.171656,0.221859,0.182361,0.042413,0.261286,0.352321,0.120567,0.254959,13.0
8,0.414610,0.356046,0.360326,0.161162,0.334040,0.236024,0.257863,0.522109,2.0
9,0.093717,0.174087,0.113384,0.135526,0.106079,0.356264,0.083088,0.189115,15.0


In [245]:
weight_result_2  # 白葡萄酒的权重

array([0.50392021, 0.05804283, 0.03013956, 0.02782145, 0.38007595])

In [247]:
Result_2 # 白葡萄酒的排名

,b*（+黄;-蓝）,总糖,可溶性固形物,干物质含量,酒石酸,正理想解,负理想解,综合得分指数,排序
0,0.152753,0.169977,0.182769,0.165614,0.148787,0.124989,0.100096,0.444703,22.0
1,0.195415,0.201768,0.192279,0.176360,0.133514,0.112990,0.126137,0.527490,16.0
2,0.179069,0.175372,0.196547,0.194878,0.290429,0.070379,0.169474,0.706576,7.0
3,0.251462,0.200901,0.191182,0.197470,0.170464,0.076415,0.170727,0.690805,9.0
4,0.265941,0.197048,0.200784,0.202565,0.226628,0.040633,0.193832,0.826700,1.0
5,0.178569,0.181056,0.179629,0.190262,0.204951,0.088202,0.133508,0.602175,12.0
6,0.194322,0.157068,0.159450,0.192963,0.104446,0.130048,0.121405,0.482813,19.0
7,0.121702,0.153215,0.159847,0.172405,0.186968,0.128795,0.094727,0.423792,23.0
8,0.185535,0.203406,0.209318,0.203649,0.253479,0.068809,0.156809,0.695020,8.0
9,0.173697,0.211691,0.200662,0.184348,0.180810,0.099760,0.123422,0.553011,14.0
